In [106]:
import yummly
import pandas as pd

# default option values
TIMEOUT = 15.0
RETRIES = 2

# Yummly mjboothaus Account: Hackathon Plan - Access granted 24 July 2017

API_ID = 'b4f167ed'
API_KEY = 'f69184af19beb4b76e7b7b1984046581'

client = yummly.Client(api_id=API_ID, api_key=API_KEY, timeout=TIMEOUT, retries=RETRIES)

In [107]:
METADATA_KEYS = [
    'ingredient',
    'holiday',
    'diet',
    'allergy',
    'technique',
    'cuisine',
    'course',
    'source',
    'brand',
    'restriction'
]

In [108]:
metadata = {i+1:key for i, key in enumerate(METADATA_KEYS)}

In [109]:
metadata_rev = dict((v, k) for k, v in metadata.items())

In [110]:
metadata_rev

{'allergy': 4,
 'brand': 9,
 'course': 7,
 'cuisine': 6,
 'diet': 3,
 'holiday': 2,
 'ingredient': 1,
 'restriction': 10,
 'source': 8,
 'technique': 5}

In [111]:
metadata

{1: 'ingredient',
 2: 'holiday',
 3: 'diet',
 4: 'allergy',
 5: 'technique',
 6: 'cuisine',
 7: 'course',
 8: 'source',
 9: 'brand',
 10: 'restriction'}

In [112]:
metadata[3]

'diet'

In [113]:
ingredient_list = client.metadata('ingredient')
diet_list = client.metadata('diet')
source_list = client.metadata('source')

meta = ['']
for i in range(1, 10):
    meta.append(client.metadata(metadata[i]))

In [114]:
#NOTE: Something weird with 'restriction' meta type -- need to check API docs

meta_dfs = ['']

for i in range(1, 10):
    print metadata[i], len(meta[i])
    meta_dfs.append(pd.DataFrame(meta[i]))

ingredient 13383
holiday 16
diet 6
allergy 10
technique 24
cuisine 26
course 13
source 77300
brand 117


In [115]:
print meta[2][0:2]

[MetaHoliday({'name': u'Christmas', 'searchValue': u'holiday^holiday-christmas', 'id': u'holiday-christmas', 'type': u'holiday', 'localesAvailableIn': [u'en-US'], 'description': u'Christmas'}), MetaHoliday({'name': u'Thanksgiving', 'searchValue': u'holiday^holiday-thanksgiving', 'id': u'holiday-thanksgiving', 'type': u'holiday', 'localesAvailableIn': [u'en-US'], 'description': u'Thanksgiving'})]


In [116]:
for i in range(1, 10):
    print str(i) + ': ' + metadata[i] + ' (' + str(len(meta[i])) + ') -----------------------------------------------'
    print meta_dfs[i].head(min(4, len(meta[i])))
    # print '--------------------------------------------------------------------'

1: ingredient (13383) -----------------------------------------------
  description searchValue    term
0        salt        salt    salt
1      butter      butter  butter
2       sugar       sugar   sugar
3      pepper      pepper  pepper
2: holiday (16) -----------------------------------------------
    description                    id localesAvailableIn          name  \
0     Christmas     holiday-christmas            [en-US]     Christmas   
1  Thanksgiving  holiday-thanksgiving            [en-US]  Thanksgiving   
2        Summer        holiday-summer            [en-US]        Summer   
3          Fall          holiday-fall            [en-US]          Fall   

                    searchValue     type  
0     holiday^holiday-christmas  holiday  
1  holiday^holiday-thanksgiving  holiday  
2        holiday^holiday-summer  holiday  
3          holiday^holiday-fall  holiday  
3: diet (6) -----------------------------------------------
    id localesAvailableIn   longDescription       

In [117]:
#type(ingredient_list)

In [118]:
ingredients = pd.DataFrame(ingredient_list)

In [119]:
metadata[metadata_rev['ingredient']]

'ingredient'

In [120]:
ingredients.head(5)

,description,searchValue,term
0,salt,salt,salt
1,butter,butter,butter
2,sugar,sugar,sugar
3,pepper,pepper,pepper
4,eggs,eggs,eggs


In [121]:
diets = pd.DataFrame(diet_list)

In [122]:
diets

,id,localesAvailableIn,longDescription,searchValue,shortDescription,type
0,388,[en-US],Lacto vegetarian,388^Lacto vegetarian,Lacto vegetarian,diet
1,389,[en-US],Ovo vegetarian,389^Ovo vegetarian,Ovo vegetarian,diet
2,390,[en-US],Pescetarian,390^Pescetarian,Pescetarian,diet
3,386,[en-US],Vegan,386^Vegan,Vegan,diet
4,387,[en-US],Vegetarian,387^Lacto-ovo vegetarian,Lacto-ovo vegetarian,diet
5,403,[en-US],Paleo,403^Paleo,Paleo,diet


In [123]:
search_params = {
    'q': '',
    'cuisine': 'American',
    'maxResult': 100000
}

In [124]:
search_params

{'cuisine': 'American', 'maxResult': 100000, 'q': ''}

In [125]:
# Start to look at recipes

#results = client.search(query_str, maxResults=100000)

results = client.search(**search_params)

print 'Total Matches:', results.totalMatchCount

#for match in results.matches:
#        print 'Recipe ID:', match.id
#        print 'Recipe:', match.recipeName
#        print 'Rating:', match.rating
#        # print 'Total Time (mins):' + match.TOTAL_TIME_MISSING / 60.0)
#        print '----------------------------------------------------'

ContentDecodingError: ('Received response with content-encoding: gzip, but failed to decode it.', error('Error -3 while decompressing: incorrect header check',))

In [32]:
type(results.matches)

list

In [33]:
len(results.matches)

40

In [34]:
results.matches[0].id

u'Chicken-marsala-308545'

In [35]:
from time import sleep

In [36]:
recipes = []
for match in results.matches:    
    this_recipe = []
    recipe = client.recipe(match.id)
    sleep(1.0)
    for ingred in recipe.ingredientLines:
        this_recipe.append(ingred)
    recipes.append(this_recipe)


In [37]:
recipes_df = pd.DataFrame(recipes)

In [38]:
recipes_df.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"3 large boneless, skinless chicken breasts (ab...","Wondra flour or all-purpose flour, for dusting",2 tablespoons canola oil,2 tablespoons unsalted butter,Kosher salt and freshly ground black pepper,Kosher salt and freshly ground black pepper,"1 pound baby bella or button mushrooms, cleane...","3 tablespoons unsalted butter, divided",1 cup dry marsala wine,"1 cup warm beef stock, preferably homemade or ...","1 tablespoon tapioca flour, optional for a thi...",Kosher salt and freshly ground black pepper,Kosher salt and freshly ground black pepper,None,None,None
1,"1¼ lb. chicken cutlets, pounded until ¼"" thick...","Kosher salt and freshly ground black pepper, t...","Kosher salt and freshly ground black pepper, t...",⅓ cup plus 1 tbsp. flour,5 tbsp. olive oil,5 tbsp. unsalted butter,8 oz. white button mushrooms,2 tbsp. minced shallots,"1 clove garlic, minced",⅓ cup dry Marsala wine,⅓ cup chicken stock,"1 tbsp. finely chopped parsley, for garnish (o...",None,None,None,None
2,1 tablespoon oil,"4 small chicken breasts, pounded thin and seas...",1 tablespoon butter,"8 ounces mushrooms, sliced","1 medium shallot, finely diced","1 clove garlic, chopped",1/2 cup marsala,1 cup chicken broth,2 tablespoons lemon juice (~1/2 lemon),salt and pepper to taste,salt and pepper to taste,1 tablespoon butter,"1 tablespoon parsley, chopped",None,None,None
3,4 boneless skinless chicken breasts (about 1 1...,1/2 cup Gold Medal™ all-purpose flour,1/4 teaspoon salt,1/4 teaspoon pepper,2 tablespoons olive or vegetable oil,"2 cloves garlic, finely chopped",1 cup sliced fresh mushrooms (3 oz),1/4 cup chopped fresh parsley or 1 tablespoon ...,1/2 cup dry Marsala wine or Progresso™ chicken...,"Hot cooked pasta, if desired",None,None,None,None,None,None
4,1/4 cup flour,Dash of white pepper,Dash of oregano,Dash of basil,1-1/4 pound boneless chicken breasts,1 tablespoon safflower oil,1 tablespoon butter or margarine,"1/2 pound mushrooms, sliced",1 cup marsala wine or dry sherry,None,None,None,None,None,None,None
5,"1 1⁄4 lb. chicken cutlets, pounded until 1⁄4"" ...","Kosher salt and freshly ground black pepper, t...","Kosher salt and freshly ground black pepper, t...",1⁄3 cup plus 1 tbsp. flour,5 tbsp. olive oil,5 tbsp. unsalted butter,8 oz. white button mushrooms,2 tbsp. minced shallots,"1 clove garlic, minced",1⁄3 cup dry Marsala wine,1⁄3 cup chicken stock,"1 tbsp. finely chopped parsley, for garnish (o...",None,None,None,None
6,1/4 cup all-purpose flour,"1 lb. boneless, skinless chicken breast halves",3 Tbsp. I Can't Believe It's Not Butter!® Spread,"1 small onion, finely chopped","1 package (10 oz.) white or cremini mushrooms,...",1 tub Knorr® Homestyle Stock - Chicken,1 cup water,1/3 cup dry Marsala wine,1/2 tsp. fresh thyme leaves (optional),None,None,None,None,None,None,None
7,1½ pounds boneless and skinless chicken breast...,1 cup flour,1 teaspoon kosher salt,½ teaspoon freshly ground black pepper,¼ cup extra virgin olive oil,4 ounces pancetta diced small,⅓ cup shallots diced,"12 ounces porcini or cremini mushrooms, quartered",½ teaspoon dry oregano,⅔ cup Marsala wine,½ cup chicken stock,3 tablespoons butter,Additional salt and pepper if needed,Additional salt and pepper if needed,"2 tablespoons fresh Italian flat leaf parsley,...",None
8,3 Tbs. all-purpose flour,"1 tsp. salt, plus more, to taste","1/2 tsp. freshly ground pepper, plus more, to ...","6 boneless, skinless chicken breast halves, ea...",3 Tbs. unsalted butter,3 Tbs. olive oil,"4 shallots, minced","1/4 lb. pancetta, diced",2 1/4 cups Marsala,1 cup chicken broth,2 tsp. dried oregano,"1/2 lb. cremini or button mushrooms, brushed c...",1/2 cup mascarpone,3 Tbs. finely chopped fresh chives,None,None
9,"4 boneless, skinless chicken breast halves (te...",1/3 cup flour,"3 tablespoons butter, divided","3 tablespoons olive oil, divided","1 medium shallot, very finely chopped (about 1...","2 cloves garlic, very finely chopped","8 ounces cremini or white butto

In [39]:
recipes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 16 columns):
0     40 non-null object
1     40 non-null object
2     40 non-null object
3     40 non-null object
4     40 non-null object
5     40 non-null object
6     40 non-null object
7     38 non-null object
8     38 non-null object
9     36 non-null object
10    29 non-null object
11    26 non-null object
12    21 non-null object
13    10 non-null object
14    7 non-null object
15    2 non-null object
dtypes: object(16)
memory usage: 5.1+ KB


In [40]:
recipes_df.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
count,40,40,40,40,40,40,40,38,38,36,29,26,21,10,7,2
unique,35,32,32,36,35,35,35,33,32,32,26,22,19,9,6,2
top,"8 ounces sliced cremini mushrooms, divided","Kosher salt and freshly ground black pepper, t...","Kosher salt and freshly ground black pepper, t...",1/4 teaspoon pepper,5 tbsp. olive oil,5 tbsp. unsalted butter,8 oz. white button mushrooms,2 tbsp. minced shallots,"1 clove garlic, minced",1/2 tablespoon all-purpose flour,1⁄3 cup chicken stock,"1 tbsp. finely chopped parsley, for garnish (o...","Italian flat-leaf parsley, for garnish",1/3 cup Marsala wine,3/4 cup reduced sodium chicken broth,1 tablespoon minced chives
freq,2,5,5,2,3,3,3,3,3,2,2,3,2,2,2,1


In [43]:
## Comment: need to standardise ingredients - see NYTimes approach
# Probably can use spaCy
# Will need to clean the text as part of this and have some sort of similarity measure for ingredients

In [44]:
type(recipes_df.iloc[0])

pandas.core.series.Series

In [45]:
import spacy
nlp = spacy.load('en')

In [58]:
def get_entity_and_type(textstr):
    textstr = unicode(textstr)
    doc = nlp(textstr)
    out_list = []
    for ent in doc.ents:
        out_list.append((ent, ent.label_, ent.label, ent.orth_)) ent.
    return out_list


# token.orth_, token.ent_type_ if token.ent_type_

In [59]:
doc1 = nlp(recipes_df[0][2])

In [67]:
recipes_df[0][2]

u'1 tablespoon oil'

In [94]:
 for ent in doc1.ents:
        for word in ent.sent:
            print word

 1
tablespoon
oil


In [90]:
for chunk in doc1.noun_chunks:
    print chunk

In [49]:
for i in range(0, 10):
    print recipes_df[0][i]

3 large boneless, skinless chicken breasts (about 2 1/2 pounds), split into 6 cutlets
1¼ lb. chicken cutlets, pounded until ¼" thick (about 8)
1 tablespoon oil
4 boneless skinless chicken breasts (about 1 1/4 lb)
1/4 cup flour
1 1⁄4 lb. chicken cutlets, pounded until 1⁄4" thick (about 8)
1/4 cup all-purpose flour
1½ pounds boneless and skinless chicken breast (2 large or 4 small)
3 Tbs. all-purpose flour
4 boneless, skinless chicken breast halves (tenderloins removed - about 1-1/4 lbs)


In [61]:
recipes_df['spacy-ingreds'] = recipes_df[0].apply(lambda ingred : get_entity_and_type(ingred))

In [62]:
print recipes_df['spacy-ingreds'][0]
print recipes_df[0][0]

[(3, u'CARDINAL', 362, u'3'), (about 2 1/2 pounds, u'MONEY', 359, u'about 2 1/2 pounds'), (6, u'CARDINAL', 362, u'6')]
3 large boneless, skinless chicken breasts (about 2 1/2 pounds), split into 6 cutlets


In [103]:
recipes_df['spacy-ingreds'].head()

0    [((3), CARDINAL, 362), ((about, 2, 1/2, pounds...
1    [((4), CARDINAL, 362), ((1, 1/4), CARDINAL, 362)]
2                              [((1¼), CARDINAL, 362)]
3                               [((1), CARDINAL, 362)]
4                               [((1), CARDINAL, 362)]
Name: spacy-ingreds, dtype: object

In [107]:
client.URL_SEARCH

'http://api.yummly.com/v1/api/recipes'

In [108]:
client.URL_BASE

'http://api.yummly.com/v1/api'

In [109]:
client.URL_GET

'http://api.yummly.com/v1/api/recipe/'

In [110]:
client.URL_META

'http://api.yummly.com/v1/api/metadata'